In [1]:
import torch.nn as nn
from torch.nn import functional as F
import torch
from models.digit_classifier import DigitClassifier
from utils import accuracy
from utils import train_model_double_objective
from importlib import reload
from models.net3 import Net3

In [2]:
from dlc_practical_prologue import generate_pair_sets

nSamples = 1000
data = generate_pair_sets(nSamples)
var_names = ["train_input", "train_target", "train_classes", "test_input", "test_target", "test_classes"]
train_input, train_target, train_classes, test_input, test_target, test_classes = data
stringWidth = 20

In [6]:
#pre-train model for classifying digits
from models.digit_classifier import DigitClassifier
import torch
from torch import nn
from utils import train_model

mini_batch_size = 100
model_classifier = DigitClassifier(out_h = 10, subnet = False)

optimizer = torch.optim.Adam(model_classifier.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()
train_in = train_input[:, 0, :, :].unsqueeze(axis = 1)
train_class = train_classes[:,0]
test_in = test_input[:, 0, :, :].unsqueeze(axis = 1)
test_class = test_classes[:,0]

train_model(model = model_classifier, 
            train_input = train_in, train_target = train_class,
            test_input = test_in, test_target = test_class,
            nb_epochs = 20, mini_batch_size = 100, 
            optimizer = optimizer, criterion = criterion,
            verbose = True)

Epoch 0: loss 34.228 / train accuracy 31.5%, test accuracy 27.6
Epoch 1: loss 17.925 / train accuracy 61.6%, test accuracy 56.1
Epoch 2: loss 12.680 / train accuracy 78.0%, test accuracy 73.3
Epoch 3: loss 8.874 / train accuracy 87.3%, test accuracy 79.6
Epoch 4: loss 5.958 / train accuracy 89.2%, test accuracy 82.8
Epoch 5: loss 4.605 / train accuracy 92.3%, test accuracy 85.4
Epoch 6: loss 3.657 / train accuracy 95.1%, test accuracy 90.1
Epoch 7: loss 2.819 / train accuracy 95.0%, test accuracy 88.9
Epoch 8: loss 2.832 / train accuracy 96.0%, test accuracy 90.9
Epoch 9: loss 2.255 / train accuracy 96.7%, test accuracy 89.2
Epoch 10: loss 1.865 / train accuracy 97.8%, test accuracy 90.4
Epoch 11: loss 1.381 / train accuracy 98.1%, test accuracy 92.3
Epoch 12: loss 1.047 / train accuracy 99.1%, test accuracy 92.2
Epoch 13: loss 0.887 / train accuracy 98.9%, test accuracy 91.7
Epoch 14: loss 0.690 / train accuracy 99.2%, test accuracy 92.6
Epoch 15: loss 0.690 / train accuracy 99.6%, te

In [8]:
model = Net3(nb_residual_blocks = 20, pretrained_submodel = model_classifier, c1 = 32, c2 = 32, c3 = 64, h = 100, p = 0.3, hidden_channel_1 = 1, hidden_channel_2 = 10)
criterion = nn.BCEWithLogitsLoss()
criterion2 = nn.CrossEntropyLoss()
eta = 1e-3
mini_batch_size = 100
optimizer = torch.optim.Adam(model.parameters(), lr = eta)
nb_epochs = 10

train_model_double_objective(model, train_input, train_target, 
                             train_classes, test_input, test_target, test_classes, 
                             nb_epochs, mini_batch_size, optimizer, criterion, criterion2, 
                             beta = 1, verbose = True)

Epoch 0: loss 16.714 / train accuracy 71.4%, test accuracy 71.7
Epoch 1: loss 10.339 / train accuracy 76.8%, test accuracy 77.4
Epoch 2: loss 8.614 / train accuracy 82.3%, test accuracy 80.9
Epoch 3: loss 7.003 / train accuracy 85.0%, test accuracy 82.6
Epoch 4: loss 6.509 / train accuracy 85.5%, test accuracy 83.7
Epoch 5: loss 6.491 / train accuracy 88.8%, test accuracy 85.2
Epoch 6: loss 5.715 / train accuracy 89.1%, test accuracy 85.9
Epoch 7: loss 5.026 / train accuracy 91.0%, test accuracy 87.8
Epoch 8: loss 4.072 / train accuracy 91.5%, test accuracy 87.8
Epoch 9: loss 4.067 / train accuracy 92.6%, test accuracy 88.5


In [9]:
from utils import accuracy_of_digit_class
print("Accuracy: {:.2f}".format(accuracy_of_digit_class(model, test_input, test_classes)))

Accuracy: 94.50


In [5]:
nParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('The model has {:d} trainable parameters'.format(nParams))

The model has 29605 trainable parameters
